In [5]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

url = "https://one.bees.com/order-management/active-orders"
emails =['leonardo.campos@rigarr.com.br','cadastro@rigarr.com.br'] 
senhas = ['Br@sil32aaaaaaa','Rigarrdistribuidora@2024'] 
filiais = ['Rigarr','Castas']

def login(email,senha):
    driver.get(url)
    campo_email = wait.until(EC.element_to_be_clickable(('id','signInName')))
    campo_email.send_keys(email)
    botao_continue = wait.until(EC.element_to_be_clickable(('id','next')))
    botao_continue.click()
    campo_senha = wait.until(EC.element_to_be_clickable(('id','password')))
    campo_senha.send_keys(senha)
    botao_continue = wait.until(EC.element_to_be_clickable(('id','next')))
    botao_continue.click()
    botao_entendi = wait.until(EC.element_to_be_clickable(('xpath','//button[text()="Entendi"]')))
    botao_entendi.click()
    time.sleep(5)
# Função para coletar dados dos pedidos
def coletar_dados_pedidos(driver):
    order_data = []
    # Espera até que a tabela de pedidos esteja visível
    tbody = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))
    )
    # Percorre todas as linhas da tabela de pedidos
    for tr in tbody.find_elements(By.XPATH, './/tr'):
        order = tr.find_elements(By.XPATH, './/td[1]')[0].text.strip()
        data_pedido = tr.find_elements(By.XPATH, './/td[2]')[0].text.strip()
        data_entrega = tr.find_elements(By.XPATH, './/td[3]')[0].text.strip()
        responsavel = tr.find_elements(By.XPATH, './/td[4]')[0].text.strip()

        # Adiciona os dados extraídos à lista
        order_data.append({
            'Numero Pedido': order,
            'Data Pedido': data_pedido,
            'Data Entrega': data_entrega,
            'Responsavel': responsavel
        })
    # Cria o DataFrame a partir dos dados extraídos
    return pd.DataFrame(order_data)

# Função para navegar pelas páginas de pedidos
def navegar_paginas(driver):
    """Navega por todas as páginas de pedidos e coleta os dados de cada uma."""
    print("Navegando pelas páginas de pedidos...")
    page_number = 1
    df_total = pd.DataFrame()  # DataFrame para armazenar todos os pedidos

    while True:
        try:
            print(f"Coletando dados da página {page_number}...")

            # Coleta dados da página atual
            df = coletar_dados_pedidos(driver)
            df_total = pd.concat([df_total, df], ignore_index=True)

            # Tenta localizar e clicar no botão "Próxima Página"
            page_xpath = f'//li[@title="page {page_number}"]'  # Ajuste o XPath conforme necessário
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, page_xpath))
            )
            next_button.click()

            # Espera até que a tabela da próxima página seja carregada
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))
            )

            # Aumenta o número da página
            page_number += 1
            time.sleep(2)  # Atraso de 2 segundos para dar tempo para a próxima página carregar

        except Exception as e:
            print(f"Não há mais páginas.")
            break

    return df_total

# Função para coletar os detalhes de cada pedido
def coletar_detalhes(driver, df):
    """Coleta os detalhes dos produtos para cada pedido, evitando duplicação de coleta."""
    detalhes = []
    pedidos_processados = set()  # Conjunto para armazenar pedidos já processados

    for order in df['Numero Pedido']:
        if order in pedidos_processados:
            continue  # Ignora o pedido se já foi processado

        try:
            print(f"Coletando detalhes para o pedido {order}...")

            # Acessa a página de detalhes do pedido
            driver.get(f'https://one.bees.com/order-management/active-orders/{order}')
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))
            )
            time.sleep(3)
            # Tenta clicar no botão 'Ver mais' se presente
            try:
                vermais_button = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/a')
                if vermais_button:
                    vermais_button[0].click()
            except Exception as e:
                continue

            # Coleta dados adicionais do pedido
            data_entrega = df[df['Numero Pedido'] == order]['Data Entrega'].values[0]
            responsavel = df[df['Numero Pedido'] == order]['Responsavel'].values[0] 

            # Coleta os dados dos produtos (caso existam)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[9]/div[3]/div/div/table/tbody'))
            )
            tbody = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[9]/div[3]/div/div/table/tbody')
            produtos = tbody.find_elements(By.XPATH, './/tr[@role="row"]')

            for produto in produtos:
                nome_sku = produto.find_element(By.XPATH, './/td[1]//div/div[2]/p[1]').text.strip()
                sku = produto.find_element(By.XPATH, './/td[1]//div/div[2]/p[2]').text.strip()
                preco = produto.find_element(By.XPATH, './/td[1]//div/div[2]/div/p').text.strip()
                quantidade_pedida = produto.find_element(By.XPATH, './/td[2]').text.strip()
                quantidade_preparar = produto.find_element(By.XPATH, './/td[3]').text.strip()

                # Coletando dados do pedido
                _numero_pedido = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[2]/div[1]/h2').text.strip().replace('Pedido #', '')
                cd = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[2]/p[2]').text.strip()
                documento = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[2]/p[2]').text.strip()

                # Coletando dados de contato e outras informações
                try:
                    ie_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[2]/p[3]')
                    ie = ie_element[0].text.strip() if ie_element else None
                except Exception as e:
                    ie_element = "ISENTO"
                    if ie_element == None:"ISENTO"
                nome_comercial = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[3]/p[2]').text.strip()
                cidade_uf = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[3]').text.strip()
                endereco = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[2]').text.strip()
                cep = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[4]').text.strip()
                _data_pedido = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[2]/div[1]/p[2]').text.strip()
                forma_pgto = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[2]/div[2]/p[2]').text.strip()
                status = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[1]/p[2]').text.strip()

                try:
                    contato1_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[2]')
                    contato1 = contato1_element[0].text.strip() if contato1_element else "teste@teste.com"
                except Exception as e:
                    contato1 = "teste@teste.com"
                    print(f"Erro ao coletar contato2 para o pedido {contato1}: {e}")
                try:

                    contato2_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[3]')
                    contato2 = contato2_element[0].text.strip() if contato2_element else None
                except Exception as e:
                    contato2 = None
                    print(f"Erro ao coletar contato2 para o pedido {contato2}: {e}")
                try:

                    contato3_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[4]')
                    contato3 = contato3_element[0].text.strip() if contato3_element else None
                except Exception as e:
                    contato3 = None
                    print(f"Erro ao coletar contato2 para o pedido {contato3}: {e}")

                detalhes.append({
                    "Documento": documento,
                    "IE": ie,
                    "Forma de Pagamento": forma_pgto,
                    "SKU": sku,
                    "Preço": preco,
                    "Quantidade Pedida": quantidade_pedida,
                    "Responsavel": responsavel,
                    "Nome Comercial": nome_comercial,
                    "Contato 1": contato1,
                    "Contato 2": contato2,
                    "Contato 3": contato3,
                    "Numero Pedido": _numero_pedido,
                    "Centro de Distribuição": cd,
                    "Status": status,
                    "Nome do Produto": nome_sku,
                    "Quantidade Preparar": quantidade_preparar,
                    "CEP": cep,
                    "Endereço de Entrega": endereco,
                    "Cidade/UF": cidade_uf,
                    "Data Pedido": _data_pedido,
                    "Data Entrega": data_entrega
                })

                pedidos_processados.add(order)  # Marca o pedido como processado

        except Exception as e:
            print(f"Erro ao coletar detalhes para o pedido {order}")
            continue
    
    return pd.DataFrame(detalhes)

for email, senha, filial in zip(emails, senhas, filiais):
    driver = webdriver.Chrome()
    driver.maximize_window()
    wait = WebDriverWait(driver, 20)

    try:
        login(email, senha)
        df_pedidos = navegar_paginas(driver)
        df_detalhes = coletar_detalhes(driver, df_pedidos)
        
        # Tratamento de contatos
        df_detalhes['Contato 1'] = df_detalhes['Contato 1'].str.replace(r"[()\s-]", "", regex=True)
        df_detalhes['Contato 2'] = df_detalhes['Contato 2'].str.replace(r"[()\s-]", "", regex=True)

        # Salvar o arquivo no caminho especificado
        caminho = r'G:\Drives compartilhados\Cadastro BEES'
        df_final = df_detalhes.to_csv(f'{caminho}/Pedidos_A_Preparar_{filial}.csv', index=False, encoding='utf-8-sig')
        print(f'Finalizado Filial {filial}')
    except Exception as e:
        print(f"Erro ao processar a filial {filial}: {e}")
    finally:
        print(f"Fechando o navegador para a filial {filial}")


Navegando pelas páginas de pedidos...
Coletando dados da página 1...
Coletando dados da página 2...
Não há mais páginas.
Coletando detalhes para o pedido 9199364476...
Coletando detalhes para o pedido 9199354851...
Coletando detalhes para o pedido 9199287732...
Finalizado Filial Rigarr
Fechando o navegador para a filial Rigarr
Navegando pelas páginas de pedidos...
Coletando dados da página 1...
Não há mais páginas.
Erro ao processar a filial Castas: 'Numero Pedido'
Fechando o navegador para a filial Castas


In [6]:
import pandas as pd
import openpyxl
from datetime import datetime
from openpyxl.styles import Font, Alignment

# Caminho para a planilha
caminho_planilha = r'G:\Drives compartilhados\OPERAÇÃO BEES\OPERAÇÃO BEES BIB.xlsx'

# Caminhos dos arquivos CSV gerados pelos pedidos
arquivo_castas = r'G:\Drives compartilhados\Cadastro BEES\Pedidos_A_Preparar_Castas.csv'
arquivo_rigarr = r'G:\Drives compartilhados\Cadastro BEES\Pedidos_A_Preparar_Rigarr.csv'

# Função para preencher a planilha
def preencher_planilha():
    # Carregar os arquivos CSV em DataFrames
    df_castas = pd.read_csv(arquivo_castas, sep=",")
    df_rigarr = pd.read_csv(arquivo_rigarr, sep=",")
    
    # Concatenar os DataFrames (juntar os dois arquivos)
    df_detalhes = pd.concat([df_castas, df_rigarr], ignore_index=True)

    # Carregar o arquivo Excel
    wb = openpyxl.load_workbook(caminho_planilha)
    ws = wb.active

    # Encontrar o índice das colunas pela primeira linha
    colunas = {cell.value: idx + 1 for idx, cell in enumerate(ws[1])}

    # Verifique se as colunas necessárias existem
    colunas_necessarias = ["DATA", "DESTINATÁRIO", "ROTA", "FORMA PAGTº", "PEDIDO"]
    for coluna in colunas_necessarias:
        if coluna not in colunas:
            raise ValueError(f"A coluna '{coluna}' não foi encontrada na planilha!")

    # Conjunto para verificar duplicatas
    dados_inseridos = set()

    # Função para formatar data no formato dd/mm/aaaa
    def formatar_data(data_str):
        try:
            # Tenta remover qualquer hora ou texto adicional
            data_str = data_str.split(',')[0]  # Remove hora ou qualquer texto extra
            # Converte para datetime, o formato de data é reconhecido pela string
            return datetime.strptime(data_str, '%d de %b. de %Y').strftime('%d/%m/%Y')
        except Exception as e:
            print(f"Erro ao formatar data: {e}")
            return None  # Caso haja erro na conversão, retorna None

    # Adicionar os números de pedidos existentes na planilha
    pedidos_existentes = set()
    for row in ws.iter_rows(min_row=2, max_col=colunas['PEDIDO'], values_only=True):
        pedidos_existentes.add(row[colunas['PEDIDO'] - 1])  # Ajuste para índice zero-based

    # Criar o objeto de fonte com "Calibri Light" e tamanho 16
    fonte = Font(name='Calibri Light', size=16)

    # Criar o objeto de alinhamento centralizado
    alinhamento = Alignment(horizontal='center', vertical='center')

    # Função para formatar a forma de pagamento
    def formatar_forma_pgto(forma_pgto):
        forma_pgto = forma_pgto.upper()  # Converte para maiúsculas

        # Substituir termos específicos
        if 'PIX NA ENTREGA' in forma_pgto:
            return 'PIX'
        elif 'CARTÃO DE CRÉDITO NA ENTREGA (SOMENTE EM 1X)' in forma_pgto:
            return 'CARTÃO DE CRÉDITO'
        elif 'CARTÃO DE DÉBITO NA ENTREGA (SOMENTE EM 1X)' in forma_pgto:
            return 'CARTÃO DE DÉBITO'
        
        return forma_pgto  # Retorna a forma de pagamento se não houver alterações

    # Preencher a planilha com os dados
    for index, row in df_detalhes.iterrows():
        cidade_uf = row['Cidade/UF']
        documento = row['Documento']  # Nome correto da coluna do CPF
        
        # Verificar se "RJ" está presente no campo 'Cidade/UF' (case insensitive) e "CPF" no campo 'Documento'
        if 'RJ' not in cidade_uf.upper() or 'CPF' not in documento.upper():  # Verifica 'RJ' e 'CPF' (case insensitive)
            continue  # Ignora se não for do estado RJ ou não for CPF
        
        # Obter dados da linha
        data_pedido = row['Data Pedido']  # Nome correto da coluna
        responsavel = row['Responsavel']  # Nome correto da coluna
        endereco = row['Endereço de Entrega']  # Nome correto da coluna
        forma_pgto = row['Forma de Pagamento']  # Nome correto da coluna
        numero_pedido = row['Numero Pedido']  # Nome correto da coluna
        
        # Verificar se o número do pedido já existe na planilha
        if numero_pedido in pedidos_existentes:
            continue  # Ignora se o número do pedido já foi inserido

        # Formatar a data para o formato dd/mm/aaaa
        data_pedido_formatada = formatar_data(data_pedido)
        
        if data_pedido_formatada is None:
            continue  # Ignora se a data não pôde ser formatada corretamente
        
        # Definir a rota com base na cidade
        cidade = cidade_uf.split(',')[0].strip()  # Extrai a cidade antes da vírgula
        if cidade.upper() == "RIO DE JANEIRO":
            rota = endereco.split(',')[-1].strip()  # Rota será o último item do endereço
        else:
            rota = cidade  # Rota será a cidade extraída do campo 'Cidade/UF'

        # Criar uma chave única para verificar duplicidade (pode incluir a combinação dos campos)
        chave_unica = (data_pedido_formatada, responsavel, rota, forma_pgto, numero_pedido)

        if chave_unica in dados_inseridos:
            continue  # Ignora se os dados já foram inseridos

        # Marcar os dados como inseridos
        dados_inseridos.add(chave_unica)

        # Encontrar a próxima linha disponível para inserir os dados
        ultima_linha = len(ws['A']) + 1  # A última linha preenchida na coluna A

        # Formatando a forma de pagamento
        forma_pgto_formatada = formatar_forma_pgto(forma_pgto)

        # Adicionar os dados na próxima linha disponível
        ws.cell(row=ultima_linha, column=colunas['DATA'], value=data_pedido_formatada).font = fonte
        ws.cell(row=ultima_linha, column=colunas['DESTINATÁRIO'], value=responsavel).font = fonte
        ws.cell(row=ultima_linha, column=colunas['ROTA'], value=rota).font = fonte
        ws.cell(row=ultima_linha, column=colunas['FORMA PAGTº'], value=forma_pgto_formatada).font = fonte
        ws.cell(row=ultima_linha, column=colunas['PEDIDO'], value=numero_pedido).font = fonte

        # Aplicar o alinhamento centralizado nas células
        for col in [colunas['DATA'], colunas['DESTINATÁRIO'], colunas['ROTA'], colunas['FORMA PAGTº'], colunas['PEDIDO']]:
            ws.cell(row=ultima_linha, column=col).alignment = alinhamento

        # Adicionar o número do pedido à lista de pedidos existentes
        pedidos_existentes.add(numero_pedido)

    # Salvar o arquivo Excel com as novas informações
    wb.save(caminho_planilha)
    print(f"Planilha {caminho_planilha} atualizada com sucesso!")

# Chame a função para preencher a planilha com os dados coletados
preencher_planilha()


Planilha G:\Drives compartilhados\OPERAÇÃO BEES\OPERAÇÃO BEES BIB.xlsx atualizada com sucesso!


In [7]:
import os
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuração do WebDriverWait
wait = WebDriverWait(driver, 30)

# Lista de cidades do interior
cidades_interior = ['ITATIAIA', 'RESENDE', 'PORTO REAL', 'QUATIS', 'BARRA MANSA', 'VOLTA REDONDA', 'ARROZAL', 'PINHEIRAL', 'PIRAI', 'BARRA DO PIRAI', 'MENDES', 'VASSOURA', 'RIO CLARO']

# Função para enviar mensagens de pedido
def enviar_mensagem_pedido(driver, grupo, pedido):
    estabelecimento = grupo['Nome Comercial'].iloc[0]
    contato1 = grupo['Contato 1'].iloc[0]
    contato2 = grupo['Contato 2'].iloc[0]
    contato3 = grupo['Contato 3'].iloc[0]
    forma_pgto = grupo['Forma de Pagamento'].iloc[0]
    endereco = grupo['Endereço de Entrega'].iloc[0]
    cidade_uf = grupo['Cidade/UF'].iloc[0]
    cep = grupo['CEP'].iloc[0]
    documento = grupo['Documento'].iloc[0]
    ie = grupo['IE'].iloc[0]
    data_entrega = grupo['Data Entrega'].iloc[0]
    responsavel = grupo['Responsavel'].iloc[0]

    mensagem = f"Número Pedido *{pedido}*\n*Responsavel*: {responsavel}\n*Estabelecimento*: {estabelecimento}\n*Contato 1*: {contato1}\n*Contato 2*: {contato2}\n*Contato 3*: {contato3}\n*Forma de Pagamento*: {forma_pgto}\n*Endereço de Entrega*: {endereco}\n*Cidade/UF*: {cidade_uf}\n*CEP*: {cep}\n{documento}\n{ie}\n*Data Entrega*: {data_entrega}\n"
    
    for _, row in grupo.iterrows():
        mensagem += f"*Produto*: {row['Nome do Produto']} | Quantidade: {row['Quantidade Pedida']} | Preço Total: R${row['Preço']}\n"

    return mensagem


# Função para verificar as condições e enviar as mensagens
def verificar_condicoes_e_enviar_mensagem(driver, df_final):
    for pedido, grupo in df_final.groupby('Numero Pedido'):
        try:
            cidade_uf = grupo['Cidade/UF'].iloc[0].strip().upper()  # Normaliza o texto
            documento = grupo['Documento'].iloc[0].strip().upper()  # Normaliza o texto

            # Depuração
            print(f"Processando pedido {pedido}: Cidade/UF = {cidade_uf}, Documento = {documento}")

            # Gera a mensagem
            mensagem = enviar_mensagem_pedido(driver, grupo, pedido)

            # Define o grupo de destino com base nas condições
            if not any(cidade in cidade_uf for cidade in cidades_interior) and "CPF" in documento and "RJ" in cidade_uf:
                grupo_destino = "NF PEDIDOS VENDAS bebida in box Bees"
            else:
                print(f"Pedido {pedido} não atende a nenhuma condição.")
                continue

            print(f"Enviando pedido {pedido} para o grupo: {grupo_destino}")

            # Localiza e interage com o campo de busca do WhatsApp
            field_find = wait.until(EC.presence_of_element_located((By.XPATH, '(//div[@role="textbox"])[1]')))
            field_find.clear()
            field_find.send_keys(grupo_destino)
            field_find.send_keys(Keys.ENTER)

            # Localiza e envia a mensagem
            field_message = wait.until(EC.presence_of_element_located((By.XPATH, '(//div[@role="textbox"])[2]')))
            field_message.send_keys(mensagem)

            botao_enviar = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Enviar']")))
            botao_enviar.click()

            print(f"Mensagem enviada para o pedido {pedido} no grupo {grupo_destino}.")
        except Exception as e:
            print(f"Erro ao enviar mensagem para o pedido {pedido}: {e}")

# Função para localizar arquivos CSV
def localizar_arquivos_csv(diretorio_base):
    arquivos = []
    for raiz, _, nomes in os.walk(diretorio_base):
        for nome in nomes:
            if nome.startswith("Pedidos_A_Preparar_") and nome.endswith(".csv"):
                arquivos.append(os.path.join(raiz, nome))
    return arquivos

# Localizando os arquivos CSV gerados dinamicamente
diretorio_base = caminho  # Substitua pelo caminho base onde os arquivos são salvos
arquivos_csv = localizar_arquivos_csv(diretorio_base)

# Verificando se os arquivos foram encontrados
if not arquivos_csv:
    print("Nenhum arquivo CSV encontrado. Verifique o caminho e o padrão dos arquivos.")
else:
    print(f"Arquivos encontrados: {arquivos_csv}")

    # Carregando os dados em um único DataFrame
    df_list = [pd.read_csv(arquivo, encoding='utf-8-sig') for arquivo in arquivos_csv]
    df_final = pd.concat(df_list, ignore_index=True)

    # Abrindo o WhatsApp Web
    driver.get("https://web.whatsapp.com/")
    while len(driver.find_elements(By.ID, "side")) < 1:
        time.sleep(1)
    time.sleep(5)
    driver.find_element('xpath','//span[@data-icon="settings-outline"]').click()
    time.sleep(3)
    driver.find_element('xpath','//div[text()="Conversas"]').click()
    time.sleep(3)
    driver.find_element('xpath','//*[@id="app"]/div/div[3]/div/div[2]/div[1]/span/div/span/div/div/div/div/div/div[2]/div[6]/div/div').click()
    time.sleep(3)
    driver.find_element('xpath','//*[@id="app"]/div/div[3]/div/header/div/div/div/div/span/div/div[1]/div[1]/button/div/span').click()
    time.sleep(3)


    # Enviando mensagens
    verificar_condicoes_e_enviar_mensagem(driver, df_final)

time.sleep(5)


Arquivos encontrados: ['G:\\Drives compartilhados\\Cadastro BEES\\Pedidos_A_Preparar_Rigarr.csv', 'G:\\Drives compartilhados\\Cadastro BEES\\Pedidos_A_Preparar_Castas.csv']


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.266)
Stacktrace:
	GetHandleVerifier [0x00007FF6104880D5+2992373]
	(No symbol) [0x00007FF61011BFD0]
	(No symbol) [0x00007FF60FFB590A]
	(No symbol) [0x00007FF60FF8F4F5]
	(No symbol) [0x00007FF6100363A7]
	(No symbol) [0x00007FF61004EE72]
	(No symbol) [0x00007FF61002F113]
	(No symbol) [0x00007FF60FFFA918]
	(No symbol) [0x00007FF60FFFBA81]
	GetHandleVerifier [0x00007FF6104E6A2D+3379789]
	GetHandleVerifier [0x00007FF6104FC32D+3468109]
	GetHandleVerifier [0x00007FF6104F0043+3418211]
	GetHandleVerifier [0x00007FF61027C78B+847787]
	(No symbol) [0x00007FF61012757F]
	(No symbol) [0x00007FF610122FC4]
	(No symbol) [0x00007FF61012315D]
	(No symbol) [0x00007FF610112979]
	BaseThreadInitThunk [0x00007FFE42D1E8D7+23]
	RtlUserThreadStart [0x00007FFE439FFBCC+44]


In [4]:
from winotify import Notification, audio

notificacao = Notification(app_id="Automacao BEES",
                     title="Automacao BEES",
                     msg="Acabou de rodar! Bora trabalhar!",
                     duration='long',
                     icon=r"C:\Users\leonardo.campos\Pictures\Imagem1.png",
                     )
notificacao.set_audio(audio.LoopingAlarm,loop=False)
notificacao.show()